# Laboratorio 4

In [2]:
import pandas as pd
from pathlib import Path

Utilizarás el conjunto de datos de pokemon.

In [3]:
pkm = (
    pd.read_csv(Path().resolve().parent / "data" / "pokemon.csv", index_col="#")
    .rename(columns=lambda x: x.replace(" ", "").replace(".", "_").lower())
)
pkm.head()

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False
4,Mega Venusaur,Grass,Poison,80,100,123,122,120,80,1,False
5,Charmander,Fire,NaN,39,52,43,60,50,65,1,False


## Ejercicio 1

(1 pto)

Agrupar por `generation` y `legendary` y obtener por grupo:

* Promedio de `hp`
* Mínimo y máximo de `sp_atk` y `sp_def`

In [4]:
(
    pkm.groupby(["generation","legendary"])
    .agg(
        hp_mean = ("hp","mean"), 
        atk_max = ("sp_atk","max"), 
        atk_min = ("sp_atk","min"),
        def_max = ("sp_def","max"),
        def_min = ("sp_def","min")
    )
)

hp_mean  atk_max  atk_min  def_max  def_min
generation legendary                                                
1          False       64.612500      175       15      130       20
           True        98.000000      194       95      125       85
2          False       69.613861      165       10      230       20
           True       103.400000      115       90      154       75
3          False       64.626761      165       10      135       20
           True        81.666667      180       50      200       20
4          False       69.740741      140       10      150       30
           True       100.846154      150       75      130       70
5          False       69.353333      145       15      135       25
           True        96.133333      170       72      129       72
6          False       66.189189      150       27      154       25
           True        87.500000      170       81      150       90

## Ejercicio 2

(1 pto)

El profesor Oakgueda determinó que una buen indicador de pokemones es: 

$$ 0.2 \, \textrm{hp} + 0.4 \,(\textrm{attack} + \textrm{sp_atk})^2 + 0.3 \,( \textrm{defense} + \textrm{sp_deff})^{1.5} + 0.1 \, \textrm{speed}$$

Según este indicador, ¿Qué grupo de pokemones (`type1`, `type2`) es en promedio mejor que el resto? 

In [5]:
def oakgueda_indicator(df):
    oakgueda_ind = 0.2*df["hp"].mean() + 0.4*(df["attack"].mean()+df["sp_atk"].mean())**2 + 0.3*(df["defense"].mean()+df["sp_def"].mean())**(1.5)+0.1*df["speed"].mean()
    return oakgueda_ind

In [13]:
pkm12 = pkm.groupby(["type1","type2"]).apply(lambda df: oakgueda_indicator(df))
pkm12.loc[lambda x: x == pkm12.max()]


type1  type2   
Bug    Electric     8095.156920
       Fighting    16050.908043
       Fire        11408.950702
       Flying       8648.456569
       Ghost        5959.055716
                       ...     
Water  Ice         11491.829231
       Poison       7331.968525
       Psychic     11920.073434
       Rock         9051.919451
       Steel       16325.896126
Length: 136, dtype: float64

__Respuesta__: En promedio, los pokemones Ground-Fire son mejores según el indicador del profesor Oakgueda.

## Ejercicio 3

(1 pto)

Define una función que escale los datos tal que, si $s$ es una columna: 

$$s\_scaled = \frac{s - \min(s)}{\max(s) - \min(s)}$$

Y luego transforma cada columna numérica agrupando por si el pokemon es legendario o no.

In [12]:
def minmax_scale(s):
    return (s-s.min())/(s.max() - s.min())

pkm.groupby("legendary").transform(lambda s: if : minmax_scale(s))



,hp,attack,defense,sp_atk,sp_def,speed,generation
#,,,,,,,
1,0.173228,0.244444,0.195556,0.333333,0.214286,0.258065,0.0
2,0.232283,0.316667,0.257778,0.424242,0.285714,0.354839,0.0
3,0.311024,0.427778,0.346667,0.545455,0.380952,0.483871,0.0
4,0.311024,0.527778,0.524444,0.678788,0.476190,0.483871,0.0
5,0.149606,0.261111,0.168889,0.303030,0.142857,0.387097,0.0
...,...,...,...,...,...,...,...
796,0.000000,0.357143,0.722222,0.347222,0.722222,0.000000,1.0
797,0.000000,0.785714,0.500000,0.763889,0.500000,0.461538,1.0
798,0.300000,0.428571,0.222222,0.694444,0.611111,0.153846,1.0


## Ejercicio 4

(1 pto)

El profesor Oakgueda necesita saber cuántos pokemones hay luego de filtrar el dataset tal que el grupo de (`type1`, `type2`) tenga en promedio un indicador (el del ejercicio #2) mayor a 40000.

In [14]:
pkm.groupby(["type1","type2"]).filter(lambda df: oakgueda_indicator(df)>40000)

,name,type1,type2,hp,attack,defense,sp_atk,sp_def,speed,generation,legendary
#,,,,,,,,,,,
425,Primal Groudon,Ground,Fire,100,180,160,150,90,90,3,True
799,Hoopa Unbound,Psychic,Dark,80,160,60,170,130,80,6,True


__Respuesta:__ Hay solo dos pokemones que cumplen esta condición.

## Bonus Track

Honestamente, ¿te causó risa el chiste del profesor Oakgueda? Responde del 1 al 5, donde 1 es equivalente a _"Me dio vergüenza ajena"_ y 5 a _"Me alegró el día y mi existencia en la UTFSM"_.

ta muy wenoxdddddd

__Respuesta__: 1